In [ ]:
import pandas as pd

data = pd.read_csv('DEC24-25-W1-W9.csv', encoding='ISO-8859-1', delimiter=';', na_values=['', 'NaN'])
df = pd.DataFrame(data)
df_clean = df.dropna(how='all')
print(df_clean)

          Event Region      Place             Location            Player Name  \
0      Regional    NaN  1st Place           Hilo Hawai     Aidan Jayke Ah Hee   
1      Regional    NaN  2nd Place           Hilo Hawai       Ryan Cade Kuamoo   
2      Regional    NaN      Top 4           Hilo Hawai  Austin Jayce H Ah Hee   
3      Regional    NaN      Top 4           Hilo Hawai    Daniela Casell Sanz   
4      Regional    NaN      Top 8           Hilo Hawai       Hunter Henderson   
...         ...    ...        ...                  ...                    ...   
12123  Regional     EU      Top 4  simbach deutschland     Jonas Dankesreiter   
12124  Regional     EU      Top 8  simbach deutschland         Matthias Fisch   
12125  Regional     EU      Top 8  simbach deutschland         Michael Kaindl   
12126  Regional     EU      Top 8  simbach deutschland          Julian Truong   
12127  Regional     EU      Top 8  simbach deutschland           Milad Ahmadi   

           Deck        Engi